# 제품 이상여부 판별 프로젝트


## 1. 데이터 불러오기


### 필수 라이브러리


In [2]:
import os
from pprint import pprint

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from tqdm import tqdm

RANDOM_STATE = 110
ROOT_DIR = "data"

### 데이터 읽어오기


In [3]:
# Load data
train_data = pd.read_csv(os.path.join(ROOT_DIR, "train.csv"))
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
data = pd.concat([train_data, test_data], axis=0)

In [8]:
THRESHOLD = len(data) / 2
data = data.dropna(thresh=THRESHOLD, axis=1)
data

,Wip Line_Dam,Process Desc._Dam,Equipment_Dam,Model.Suffix_Dam,Workorder_Dam,Insp. Seq No._Dam,Insp Judge Code_Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,...,Head Clean Position Z Collect Result_Fill2,Head Purge Position X Collect Result_Fill2,Head Purge Position Y Collect Result_Fill2,Head Purge Position Z Collect Result_Fill2,Machine Tact time Collect Result_Fill2,PalletID Collect Result_Fill2,Production Qty Collect Result_Fill2,Receip No Collect Result_Fill2,WorkMode Collect Result_Fill2,target
0,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,4F1XA938-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,114.612,19.9,7,127,1,Normal
1,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334505,3KPM0016-2,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.600,7.0,185,1,0,Normal
2,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4E1X9167-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,114.612,19.8,10,73,1,Normal
3,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1X0057-1,1,OK,1000.0,12.5,90,...,91.8,270.0,50,85,19.900,12.0,268,1,0,Normal
4,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3HPM0007-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,19.700,8.0,121,1,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17356,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,3K1XB597-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,85.000,19.5,14,131,1,NaN
17357,IVI-OB6,Dam Dispenser,Dam dispenser #2,AJX75334501,4A1XB974-1,1,OK,1000.0,12.5,90,...,50.0,91.8,270,50,85.000,19.8,12,279,1,NaN
17358,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3L1XA998-1,1,OK,240.0,2.5,-90,...,50.0,91.8,270,50,85.000,20.5,4,66,1,NaN
17359,IVI-OB6,Dam Dispenser,Dam dispenser #1,AJX75334501,3F1XC376-1,1,OK,240.0,2.5,-90,...,91.8,270.0,50,85,18.900,1.0,117,1,0,NaN


In [4]:
# data.to_csv('concat.csv')

In [9]:
data = pd.read_csv('concat.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_19604\1771210974.py:1: DtypeWarning: Columns (181) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('df.csv')


In [12]:
l = ['Dam', 'Fill1', 'Fill2']

def selection(process):
    start_col = f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{process}'
    data[start_col] = data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
    
    end_col = f'WorkMode Collect Result_{process}'
    
    selected_columns = data.loc[:, start_col:end_col]
    return selected_columns, start_col, end_col

def shifting(df, column):
    df_shifted = df.copy()
    nan_rows = df[df[column].isna()]
    
    for index in nan_rows.index:
        row = df_shifted.loc[index]
        shifted_row = row.shift(-1).fillna(pd.NA)
        df_shifted.loc[index] = shifted_row
    
    return df_shifted

for i in l:
    selected_columns_i, start_col_i, end_col_i = selection(i)
    df_shifted = shifting(selected_columns_i, start_col_i)
    data.loc[:, start_col_i:end_col_i] = df_shifted

data.to_csv('concat_mod.csv', index=False)

In [13]:
data = pd.read_csv("concat_mod.csv")
X = data.drop(columns=['target'])
y = data['target']

C:\Users\USER\AppData\Local\Temp\ipykernel_19604\1147735921.py:1: DtypeWarning: Columns (181) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("concat_mod.csv")


In [14]:
from sklearn.preprocessing import LabelEncoder

non_numeric_columns = X.select_dtypes(include=['object']).columns

encoder = LabelEncoder()
encoded_df = pd.DataFrame(index=X.index)
for column in non_numeric_columns:
    encoded_df[column] = encoder.fit_transform(X[column])

X = X.drop(columns=non_numeric_columns)
X = pd.concat([X, encoded_df], axis=1)
X

,Unnamed: 0,Insp. Seq No._Dam,CURE END POSITION X Collect Result_Dam,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION X Collect Result_Dam,CURE STANDBY POSITION Z Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION X Collect Result_Dam,...,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2
0,0,1,240.0,2.5,-90,100,1150,33.5,0,1030,...,0,3,658,0,0,0,0,3,658,0
1,1,1,240.0,2.5,-90,70,1150,33.5,0,1030,...,0,3,284,0,0,0,0,3,284,0
2,2,1,1000.0,12.5,90,85,1150,33.5,0,280,...,1,0,590,0,0,0,1,0,590,0
3,3,1,1000.0,12.5,90,70,1150,33.5,0,280,...,1,0,252,0,0,0,1,0,252,0
4,4,1,240.0,2.5,-90,70,1150,33.5,0,1030,...,0,0,143,0,0,0,0,0,143,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57862,17356,1,1000.0,12.5,90,70,1150,33.5,0,280,...,1,0,278,0,0,0,1,0,278,0
57863,17357,1,1000.0,12.5,90,70,1150,33.5,0,280,...,1,0,440,0,0,0,1,0,440,0
57864,17358,1,240.0,2.5,-90,70,1150,33.5,0,1030,...,0,0,315,0,0,0,0,0,315,0
57865,17359,1,240.0,2.5,-90,70,1150,33.5,0,1030,...,0,0,8,0,0,0,0,0,8,0


In [15]:
coordinate_columns = [
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Dam"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Dam"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Dam", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Dam"),
    ("HEAD Standby Position X Collect Result_Dam", "HEAD Standby Position Y Collect Result_Dam", "HEAD Standby Position Z Collect Result_Dam"),
    ("Head Clean Position X Collect Result_Dam", "Head Clean Position Y Collect Result_Dam", "Head Clean Position Z Collect Result_Dam"),
    ("Head Purge Position X Collect Result_Dam", "Head Purge Position Y Collect Result_Dam", "Head Purge Position Z Collect Result_Dam"),
    ("Head Zero Position X Collect Result_Dam", "Head Zero Position Y Collect Result_Dam", "Head Zero Position Z Collect Result_Dam"),
    
    ("HEAD Standby Position X Collect Result_Fill1", "HEAD Standby Position Y Collect Result_Fill1", "HEAD Standby Position Z Collect Result_Fill1"),
    ("Head Clean Position X Collect Result_Fill1", "Head Clean Position Y Collect Result_Fill1", "Head Clean Position Z Collect Result_Fill1"),
    ("Head Purge Position X Collect Result_Fill1", "Head Purge Position Y Collect Result_Fill1", "Head Purge Position Z Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill1"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill1", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill1"),
    
    ("HEAD Standby Position X Collect Result_Fill2", "HEAD Standby Position Y Collect Result_Fill2", "HEAD Standby Position Z Collect Result_Fill2"),
    ("Head Clean Position X Collect Result_Fill2", "Head Clean Position Y Collect Result_Fill2", "Head Clean Position Z Collect Result_Fill2"),
    ("Head Purge Position X Collect Result_Fill2", "Head Purge Position Y Collect Result_Fill2", "Head Purge Position Z Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage1) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage1) Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage2) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage2) Collect Result_Fill2"),
    ("HEAD NORMAL COORDINATE X AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Y AXIS(Stage3) Collect Result_Fill2", "HEAD NORMAL COORDINATE Z AXIS(Stage3) Collect Result_Fill2"),
]

cylindrical_columns = [
    ("CURE START POSITION X Collect Result_Dam", "CURE START POSITION Z Collect Result_Dam", "CURE START POSITION Θ Collect Result_Dam"),
    ("CURE STANDBY POSITION X Collect Result_Dam", "CURE STANDBY POSITION Z Collect Result_Dam", "CURE STANDBY POSITION Θ Collect Result_Dam"),
    ("CURE END POSITION X Collect Result_Dam", "CURE END POSITION Z Collect Result_Dam", "CURE END POSITION Θ Collect Result_Dam"),
    ("CURE START POSITION X Collect Result_Fill2", "CURE START POSITION Z Collect Result_Fill2", "CURE START POSITION Θ Collect Result_Fill2"),
    ("CURE STANDBY POSITION X Collect Result_Fill2", "CURE STANDBY POSITION Z Collect Result_Fill2", "CURE STANDBY POSITION Θ Collect Result_Fill2"),
    ("CURE END POSITION X Collect Result_Fill2", "CURE END POSITION Z Collect Result_Fill2", "CURE END POSITION Θ Collect Result_Fill2"),
]

for x_col, y_col, z_col in coordinate_columns:
    
    X[x_col] = np.sqrt(X[x_col]**2 + X[y_col]**2 + X[z_col]**2)
    X[y_col] = np.arctan2(X[y_col], X[x_col])
    X[z_col] = np.arccos(X[z_col] / X[x_col])
    
    r_col = x_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    theta_col = y_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    phi_col = z_col.replace("X", "r").replace("Y", "θ").replace("Z", "φ")
    
    X.rename(columns={x_col: r_col, y_col: theta_col, z_col: phi_col}, inplace=True)

for x_col, z_col, theta_col in cylindrical_columns:
    
    X[x_col] = np.sqrt(X[x_col]**2 + X[z_col]**2)
    # df[theta_col] = df[theta_col]
    X[z_col] = np.arctan2(X[x_col], X[z_col])
    
    r_col = x_col.replace("X", "r").replace("Z", "φ").replace("Θ", "θ")
    phi_col = z_col.replace("X", "r").replace("Z", "φ").replace("Θ", "θ")
    
    X.rename(columns={x_col: r_col, z_col: phi_col}, inplace=True)

X.head()

,Unnamed: 0,Insp. Seq No._Dam,CURE END POSITION r Collect Result_Dam,CURE END POSITION φ Collect Result_Dam,CURE END POSITION Θ Collect Result_Dam,CURE SPEED Collect Result_Dam,CURE STANDBY POSITION r Collect Result_Dam,CURE STANDBY POSITION φ Collect Result_Dam,CURE STANDBY POSITION Θ Collect Result_Dam,CURE START POSITION r Collect Result_Dam,...,Equipment_Fill1,Model.Suffix_Fill1,Workorder_Fill1,Insp Judge Code_Fill1,Wip Line_Fill2,Process Desc._Fill2,Equipment_Fill2,Model.Suffix_Fill2,Workorder_Fill2,Insp Judge Code_Fill2
0,0,1,240.013020,1.560381,-90,100,1150.487831,1.541686,0,1030.544638,...,0,3,658,0,0,0,0,3,658,0
1,1,1,240.013020,1.560381,-90,70,1150.487831,1.541686,0,1030.544638,...,0,3,284,0,0,0,0,3,284,0
2,2,1,1000.078122,1.558298,90,85,1150.487831,1.541686,0,281.996897,...,1,0,590,0,0,0,1,0,590,0
3,3,1,1000.078122,1.558298,90,70,1150.487831,1.541686,0,281.996897,...,1,0,252,0,0,0,1,0,252,0
4,4,1,240.013020,1.560381,-90,70,1150.487831,1.541686,0,1030.544638,...,0,0,143,0,0,0,0,0,143,0


In [16]:
df = pd.concat([X, y], axis=1)

In [17]:
df.to_csv('concat_sph.csv', index=False)

In [23]:
single_unique = [col for col in df.columns if df[col].nunique() == 1]
df = df.drop(single_unique, axis=1)

numeric_cols = df.select_dtypes(include=[int, float]).columns
concat_cols = numeric_cols.union(['target'])

df = df[concat_cols]
df

,1st Pressure 1st Pressure Unit Time_AutoClave,1st Pressure Collect Result_AutoClave,2nd Pressure Collect Result_AutoClave,2nd Pressure Unit Time_AutoClave,3rd Pressure Collect Result_AutoClave,3rd Pressure Unit Time_AutoClave,CURE END POSITION X Collect Result_Dam,CURE END POSITION X Collect Result_Fill2,CURE END POSITION Z Collect Result_Dam,CURE END POSITION Z Collect Result_Fill2,...,Stage3 Circle3 Distance Speed Collect Result_Dam,Stage3 Circle4 Distance Speed Collect Result_Dam,Stage3 Line1 Distance Speed Collect Result_Dam,Stage3 Line2 Distance Speed Collect Result_Dam,Stage3 Line3 Distance Speed Collect Result_Dam,Stage3 Line4 Distance Speed Collect Result_Dam,THICKNESS 1 Collect Result_Dam,THICKNESS 2 Collect Result_Dam,THICKNESS 3 Collect Result_Dam,target
0,240,0.312,0.493,1,0.499,120,240.0,240,2.5,33,...,5800,5800,5800,5800,5800,5800,0.000,0.000,0.000,Normal
1,241,0.311,0.311,121,0.498,121,240.0,240,2.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
2,241,0.315,0.490,1,0.498,121,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,0.012,-0.022,0.003,Normal
3,241,0.299,0.300,121,0.500,121,1000.0,240,12.5,33,...,4000,4000,4000,4000,4000,4000,0.000,0.000,0.000,Normal
4,240,0.302,0.302,120,0.498,120,240.0,240,2.5,32,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40501,241,0.300,0.300,121,0.500,121,240.0,240,2.5,33,...,5000,5000,5000,5000,5000,5000,0.000,0.000,0.000,Normal
40502,240,0.305,0.493,1,0.498,120,1000.0,240,12.5,33,...,5800,5800,5800,5800,5800,5800,-0.019,-0.021,-0.118,Normal
40503,241,0.309,0.490,1,0.498,121,240.0,240,2.5,33,...,6000,6000,6000,6000,6000,6000,0.000,0.000,0.000,Normal
40504,241,0.302,0.303,121,0.499,121,1000.0,240,12.5,33,...,9000,9000,9000,9000,9000,9000,0.000,0.000,0.000,Normal


In [10]:
# df_concat = train_data

### 언더 샘플링


데이타 불균형을 해결하기 위해 언더 샘플링을 진행합니다.


In [11]:
normal_ratio = 1.0  # 1.0 means 1:1 ratio

df_normal = train_data[train_data["target"] == "Normal"]
df_abnormal = train_data[train_data["target"] == "AbNormal"]

num_normal = len(df_normal)
num_abnormal = len(df_abnormal)
print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}")

df_normal = df_normal.sample(n=int(num_abnormal * normal_ratio), replace=False, random_state=RANDOM_STATE)
df_concat = pd.concat([df_normal, df_abnormal], axis=0).reset_index(drop=True)
df_concat.value_counts("target")

  Total: Normal: 38156, AbNormal: 2350


target
AbNormal    2350
Normal      2350
Name: count, dtype: int64

### 데이터 분할


In [12]:
df_train, df_val = train_test_split(
    df_concat,
    test_size=0.2,
    stratify=df_concat["target"],
    random_state=RANDOM_STATE,
)

def print_stats(df: pd.DataFrame):
    num_normal = len(df[df["target"] == "Normal"])
    num_abnormal = len(df[df["target"] == "AbNormal"])

    print(f"  Total: Normal: {num_normal}, AbNormal: {num_abnormal}" + f" ratio: {num_abnormal/num_normal}")

# Print statistics
print(f"  \tAbnormal\tNormal")
print_stats(df_train)
print_stats(df_val)

  	Abnormal	Normal
  Total: Normal: 1880, AbNormal: 1880 ratio: 1.0
  Total: Normal: 470, AbNormal: 470 ratio: 1.0


## 3. 모델 학습


### 모델 정의


In [13]:
model = RandomForestClassifier(random_state=RANDOM_STATE)

### 모델 학습


In [14]:
# features = []

# for col in df_train.columns:
#     try:
#         df_train[col] = df_train[col].astype(int)
#         features.append(col)
#     except:
#         continue

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

model.fit(train_x, train_y)

RandomForestClassifier(random_state=110)

In [15]:
test_x = df_val.drop(columns=['target'])

test_y = df_val["target"]
pred_y = model.predict(test_x)

f1 = f1_score(test_y, pred_y, pos_label='Normal')

print(f"F1 Score: {f1:.4f}")

F1 Score: 0.5626


In [11]:
pred_y

array(['Normal', 'Normal', 'Normal', ..., 'Normal', 'Normal', 'Normal'],
      dtype=object)

## 4. 제출하기


### 테스트 데이터 예측


테스트 데이터 불러오기


In [53]:
test_data = pd.read_csv(os.path.join(ROOT_DIR, "test.csv"))
THRESHOLD = len(test_data) / 2
test_data = test_data.dropna(thresh=THRESHOLD, axis=1)

l = ['Dam', 'Fill1', 'Fill2']

def selection(process):
    start_col = f'HEAD NORMAL COORDINATE X AXIS(Stage1) Collect Result_{process}'
    test_data[start_col] = test_data[start_col].apply(lambda x: np.nan if x == 'OK' else x)
    
    end_col = f'WorkMode Collect Result_{process}'
    
    selected_columns = test_data.loc[:, start_col:end_col]
    return selected_columns, start_col, end_col

def shifting(df, column):
    df_shifted = df.copy()
    nan_rows = df[df[column].isna()]
    
    for index in nan_rows.index:
        row = df_shifted.loc[index]
        shifted_row = row.shift(-1).fillna(pd.NA)
        df_shifted.loc[index] = shifted_row
    
    return df_shifted

for i in l:
    selected_columns_i, start_col_i, end_col_i = selection(i)
    df_shifted = shifting(selected_columns_i, start_col_i)
    test_data.loc[:, start_col_i:end_col_i] = df_shifted
    
test_data.to_csv('test_mod.csv', index=False)
test_data = pd.read_csv("test_mod.csv")

single_unique = [col for col in test_data.columns if test_data[col].nunique() == 1]
test_data = test_data.drop(single_unique, axis=1)

numeric_cols = test_data.select_dtypes(include=[int, float]).columns

test_data = test_data[numeric_cols]

model = RandomForestClassifier(random_state=RANDOM_STATE)

train_x = df_train.drop(columns=['target'])
train_y = df_train["target"]

common_cols = train_x.columns.intersection(test_data.columns)

train_x = train_x[common_cols]
test_x = test_data[common_cols]

model.fit(train_x, train_y)

pred_y = model.predict(test_x)

df_sub = pd.read_csv("submission.csv")
df_sub["target"] = pred_y

df_sub.to_csv("submission.csv", index=False)

ValueError: Length of values (40506) does not match length of index (17361)

In [ ]:
# df_test_x = test_data[features]

# for col in df_test_x.columns:
#     try:
#         df_test_x.loc[:, col] = df_test_x[col].astype(int)
#     except:
#         continue

In [ ]:
# test_pred = model.predict(df_test_x)
# test_pred

### 제출 파일 작성


In [ ]:
# # 제출 데이터 읽어오기 (df_test는 전처리된 데이터가 저장됨)
# df_sub = pd.read_csv("submission.csv")
# df_sub["target"] = test_pred

# # 제출 파일 저장
# df_sub.to_csv("submission.csv", index=False)

**우측 상단의 제출 버튼을 클릭해 결과를 확인하세요**
